In [1]:
#!pip install optuna

In [2]:
#!pip install qiskit

In [3]:
#!pip install qiskit_machine_learning

In [13]:
import optuna
from sklearn import model_selection
import matplotlib.pyplot as plt
import numpy as np
import math

from sklearn.svm import *
from sklearn.cluster import *
from sklearn.metrics import *
from sklearn.decomposition import *
from sklearn.preprocessing import MinMaxScaler

from qiskit import *
from qiskit.circuit.library import *
from qiskit.utils import *
from qiskit_machine_learning.algorithms import *
from qiskit_machine_learning.kernels import *
from qiskit_machine_learning.datasets import *
import warnings
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

warnings.filterwarnings("ignore")
from qiskit_algorithms.utils import algorithm_globals

seed = 33
np.random.seed(seed)
algorithm_globals.random_seed = seed


In [8]:
from qiskit_machine_learning.datasets import ad_hoc_data

adhoc_dimension = 3
train_features, train_labels, test_features, test_labels, adhoc_total = ad_hoc_data(
    training_size=50,
    test_size=20,
    n=adhoc_dimension,
    gap=0.25,
    plot_data=False,
    one_hot=False,
    include_sample_total=True,
)


In [70]:
def objective(trial):
    """
    Objective function for Optuna optimization.

    Parameters:
    -----------
    trial : `optuna.Trial` object
        A single optimization trial. It is used to sample hyperparameters.

    Returns:
    --------
    float
        The maximum mean accuracy achieved by the best performing quantum kernel among ZZ and Z kernels.

    Notes:
    ------
    This function is designed to be used as an objective function for hyperparameter optimization using Optuna.
    It takes hyperparameters to optimize, including the number of repetitions in the feature map (`rep`),
    the regularization parameter for the SVC classifier (`c`), the type of entanglement in the feature map (`entanglement`),
    and the number of shots for quantum circuit execution (`shots`).

    The function creates ZZ and Z feature maps based on the specified hyperparameters, defines a fidelity computation method using
    the 'ComputeUncompute' class, creates quantum kernels using the feature maps and fidelity, evaluates the kernels
    on the training data, performs cross-validation with SVM classifiers using precomputed kernels, and computes the mean
    accuracy of the best performing kernel among ZZ and Z kernels.

    The maximum mean accuracy achieved by the best performing kernel is returned as the optimization target.
    """

    # Define hyperparameters to optimize

    # Number of repetitions in feature map
    rep = trial.suggest_int("rep", 1, 5)

    # Regularization parameter for the SVC classifier
    c = trial.suggest_float("C", 0.1, 1000)

    # Type of entanglement in the feature map
    entangle = trial.suggest_categorical("entanglement", ["full", "linear"])

    # Number of shots for quantum circuit execution
    shots = trial.suggest_int("shots", 100, 10000)

    # Number of features in the data
    num_qubits = train_features.shape[1]

    # Define the feature maps

    # ZZ Feature Map with specified parameters
    zz_feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=rep, entanglement=entangle)

    # Z Feature Map with specified parameters
    z_feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=rep)

    # Define the quantum sampler

    # Quantum sampler with specified number of shots
    sampler = Sampler(options={"shots": shots})

    # Define the fidelity

    # Fidelity computation method using the defined sampler
    fidelity = ComputeUncompute(sampler=sampler)

    # Define the quantum kernels

    # Quantum kernel using ZZ Feature Map and fidelity
    zz_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=zz_feature_map)

    # Quantum kernel using Z Feature Map and fidelity
    z_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=z_feature_map)

    # Evaluate each quantum kernel on the training data

    # Gram matrix for ZZ kernel
    zz_gram_matrix = zz_kernel.evaluate(x_vec=train_features, y_vec=train_features)

    # Gram matrix for Z kernel
    z_gram_matrix = z_kernel.evaluate(x_vec=train_features, y_vec=train_features)

    # Define the classifiers using precomputed kernels

    # SVM classifier using precomputed kernel with specified regularization parameter
    zz_classifier = SVC(kernel="precomputed", shrinking=True, C=c, random_state=seed)

    # SVM classifier using precomputed kernel with specified regularization parameter
    z_classifier = SVC(kernel="precomputed", shrinking=True, C=c, random_state=seed)

    # Perform cross-validation with each classifier

    # Cross-validation scores for ZZ kernel classifier
    zz_scores = model_selection.cross_val_score(zz_classifier, zz_gram_matrix, train_labels, n_jobs=10, cv=5)

    # Cross-validation scores for Z kernel classifier
    z_scores = model_selection.cross_val_score(z_classifier, z_gram_matrix, train_labels, n_jobs=10, cv=5)

    # Calculate the mean accuracy for each kernel

    # Mean accuracy for ZZ kernel
    zz_accuracy = zz_scores.mean()

    # Mean accuracy for Z kernel
    z_accuracy = z_scores.mean()

    # Return the mean accuracy of the best performing kernel

    # Return the maximum accuracy among ZZ and Z kernels
    return np.max([zz_accuracy, z_accuracy])


In [62]:
study = optuna.create_study(direction="maximize",study_name='SVC_AbdoMostafa')# internally it uses the TPE samplerhttps://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.TPESampler.html
study.optimize(objective, timeout= 10 * 60)# running this for more than 10 minutes might get better results with increasing the search space of course

[I 2024-05-09 22:15:36,488] A new study created in memory with name: SVC_AbdoMostafa
[I 2024-05-09 22:16:34,623] Trial 0 finished with value: 1.0 and parameters: {'rep': 2, 'C': 418.79467881122827, 'entanglement': 'full', 'shots': 8078}. Best is trial 0 with value: 1.0.
[I 2024-05-09 22:17:56,334] Trial 1 finished with value: 0.67 and parameters: {'rep': 4, 'C': 239.88632532607107, 'entanglement': 'linear', 'shots': 5730}. Best is trial 0 with value: 1.0.
[I 2024-05-09 22:19:17,163] Trial 2 finished with value: 0.62 and parameters: {'rep': 4, 'C': 940.5861275211739, 'entanglement': 'full', 'shots': 3768}. Best is trial 0 with value: 1.0.
[I 2024-05-09 22:19:48,198] Trial 3 finished with value: 0.76 and parameters: {'rep': 1, 'C': 376.7795639663311, 'entanglement': 'full', 'shots': 3383}. Best is trial 0 with value: 1.0.
[I 2024-05-09 22:20:20,536] Trial 4 finished with value: 0.6 and parameters: {'rep': 1, 'C': 380.0333155497777, 'entanglement': 'linear', 'shots': 1947}. Best is trial 

In [63]:
print(f"The best trial is : \n{study.best_trial}")
print()
print(f"The best value is : \n{study.best_value}")
print()
print(f"The best parameters are : \n{study.best_params}")


The best trial is : 
FrozenTrial(number=0, state=TrialState.COMPLETE, values=[1.0], datetime_start=datetime.datetime(2024, 5, 9, 22, 15, 36, 504647), datetime_complete=datetime.datetime(2024, 5, 9, 22, 16, 34, 623218), params={'rep': 2, 'C': 418.79467881122827, 'entanglement': 'full', 'shots': 8078}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'rep': IntDistribution(high=5, log=False, low=1, step=1), 'C': FloatDistribution(high=1000.0, log=False, low=0.1, step=None), 'entanglement': CategoricalDistribution(choices=('full', 'linear')), 'shots': IntDistribution(high=10000, log=False, low=100, step=1)}, trial_id=0, value=None)

The best value is : 
1.0

The best parameters are : 
{'rep': 2, 'C': 418.79467881122827, 'entanglement': 'full', 'shots': 8078}


In [64]:
# Extracting the best hyperparameters from the Optuna study
rep = study.best_params['rep']  # Number of repetitions
num_qubits = train_features.shape[1]  # Number of qubits
c = study.best_params['C']  # Regularization parameter
entangle = study.best_params['entanglement']  # Entanglement structure
shots = study.best_params['shots']  # Number of shots

# Initializing the ZZFeatureMap with the best hyperparameters
feature_map = ZZFeatureMap(feature_dimension=num_qubits,
                           reps=rep, entanglement=entangle)

# Initializing the sampler for executing quantum circuits
sampler = Sampler(options={"shots":shots})

# Initializing the fidelity estimator for quantum kernel evaluation
fidelity = ComputeUncompute(sampler=sampler)

# Initializing the FidelityQuantumKernel with the best hyperparameters
adhoc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

# Evaluating the quantum kernel on the training data
gram_train = adhoc_kernel.evaluate(x_vec=train_features, y_vec=train_features)

# Evaluating the quantum kernel on the test data
gram_test = adhoc_kernel.evaluate(x_vec=test_features, y_vec=train_features)

# Initializing the SVM classifier with precomputed kernel
classifier_obj = SVC(kernel="precomputed", shrinking=True, C=c, random_state=seed)

# Fitting the SVM classifier on the training data
classifier_obj.fit(gram_train, train_labels)

# Calculating and printing the training accuracy
train_accuracy = classifier_obj.score(gram_train, train_labels)
print("Training accuracy:", train_accuracy)

# Calculating and printing the test accuracy
test_accuracy = classifier_obj.score(gram_test, test_labels)
print("Test accuracy:", test_accuracy)


training accurcy 1.0
test accurcy 1.0


In [65]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [66]:
optuna.visualization.plot_parallel_coordinate(study, params=['rep', 'entanglement', "shots", "C"])

In [67]:
optuna.visualization.plot_param_importances(study)

In [68]:
history_csv = study.trials_dataframe()

In [69]:
history_csv.to_csv("hist.csv")